# Quantum Key Distribution: Assignment

 ### <font color='red'> Note: </font>
- It would be useful to review the previous assignment to understand the context
- You are allowed to change the noise level in the communication channel
- It would be helpful to try out certain aspects in this notebook on a pen and paper


In the previous assignment on Quantum Cryptography, we saw that the key was a very valuable commodity in secure communication. 

<font color='yellow'>**Quantum key distribution exploits the principles of quantum theory to establish a secret key between  two distant parties whose security is guaranteed by the laws of physics.**</font>

- The **BB84 protocol** developed by Charles Bennett and Gilles Brassard in $1984$ is one of the first examples of a quantum key distribution protocol.

Two authorized parties, Anshu and Bharat, want to establish a secret key over a distance. To accomplish this task, they have access to two channels:

- Quantum channel ($Q$) that allows Anshu and Bharat to send quantum signals (qubits).
(An adversary can perform any operation on the information that is sent through this channel and hence it is insecure).
- Classical channel ($C$) that Anshu and Bharat can use to send classical bits back and forth. 

![BB84](Images/BB84.png)

## BB84 Protocol (Noise-free version)

### Stage $1$: Distributing Quantum States

The key idea involved here (as was also seen in the previous assignment):

***Measuring qubit with incorrect basis randomizes the outcome!***


There are two bases that are most commonly used in QKD:

> Orthogonal states $\ket{0}$ and $\ket{1}$ form a <b>Standard Basis</b>, also called the $Z$ basis.
> Orthogonal states $\ket{+}$ and $\ket{-}$ form a <b>Hadamard Basis</b>, also called the $X$ basis.

#### Protocol for Stage 1:

<b>Anshu</b>,

- Creates a $n$ -bit long random bit string and initializes qubits accordingly.
- Encodes qubits in a basis ($X$ or $Z$) chosen randomly (does nothing on the qubit if $Z$-basis is chosen, applies $H$-gate if $X$-basis is chosen).
- Saves information about initial bit values and the corresponding basis she used to encode the bits.
- Sends qubits to Bharat through the Quantum Channel.

    
**Bharat**,
- Randomly and independently from Anshu, chooses a basis in which he will measure each received qubit.
- Measures qubits in chosen basis ($X$ or $Z$) (does nothing on the qubit if $Z$-basis is chosen, applies $H$-gate if $X$-basis is chosen).
- Saves information about the basis he used and the corresponding measurement results.

<font style="color:darkorange;font-weight:bold;"> <h3>Task 1</h3> </font>Complete the code below to implement the stage 1 of the noise-free BB84 protocol for a $16$- bit long string

In [6]:
# defining some necessary functions that can be used later

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from random import randrange

def print_outcomes_in_reserve(counts): # takes a dictionary variable
    for outcome in counts: 
        reverse_outcome = ''
        for i in outcome: 
            reverse_outcome = i + reverse_outcome 
    return reverse_outcome

#Source for SendState: awards/teach_me_qiskit_2018/cryptography/Cryptography.ipynb (slightly modified for this assignment)

#this code has been modified to include circuits with any register names as opposed to the code in the previous assignment

def SendState(qc1, qc2, qc1_name):
    ''' This function takes the output of a circuit qc1 (made up only of x and 
        h gates and initializes another circuit qc2 with the same state
    ''' 
    
    # Quantum state is retrieved from qasm code of qc1
    qs = qc1.qasm().split(sep=';')[4:-1]

    # Process the code to get the instructions
    for index, instruction in enumerate(qs):
        qs[index] = instruction.lstrip()

    for instruction in qs:
        if instruction[0] == 'x':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.x(qc2.qregs[0][old_qr])
        elif instruction[0] == 'h':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.h(qc2.qregs[0][old_qr])
        elif instruction[0] == 'm': # exclude measuring:
            pass
        else:
            raise Exception('Unable to parse instruction')

In [ ]:
import qiskit
from qiskit import *

qreg = QuantumRegister(16) # quantum register with 16 qubits
creg = ClassicalRegister(16) # classical register with 16 bits

# Quantum circuit for Anshu's state
anshu = QuantumCircuit(qreg, creg, name='anshu')

send=[] #Initial bit string to send
anshu_basis=[] # save information about encoding basis
bharat_basis=[] # save information about decoding basis

#Creating random 16-bit long string and store the values in the list 'send'

    #your code here
    
#Prepare Anshu's qubits as per the bit string generated

    #your code here


#Encoding

# randomly pick a basis to encode the qubits and also
#store the basis picked in the list anshu_basis
    
    #your code here


bharat = QuantumCircuit(qreg, creg, name='bharat') #Defining Bharat's circuit
SendState(anshu, bharat, 'anshu') #Anshu sends states to Bharat

#Bharat measures qubits

#Bharat randomly picks a basis and performs measurement in the basis picked. 
# Also store the basis picked in the list bharat_basis
 
    #your code here


job = execute(bharat,Aer.get_backend('qasm_simulator'),shots=1) #Notice that Bharat only has one shot to measure qubits
counts = job.result().get_counts(bharat) 
counts = print_outcomes_in_reserve(counts)

#Saving Bharat's received string as a list
received = list(map(int, counts))

print("Anshu sent:", send)
print("Anshu's encoding basis:", anshu_basis)
print("Bharat received:", received)
print("Bharat's decoding basis:", bharat_basis)

Ofcourse, 'Anshu sent' is not the same as 'Bharat received', as expected. 
The next stage of BB84 protocol is to remove these incorrect results (errors).

### Stage $2$: Classical Post-Processing

There are mainly three processes here: **Sifting, Error-correction, Privacy Amplification.**

#### Sifting

Anshu and Bharat should determine, by *public exchange* of messages (through classical channel), the rounds in which their encoding and decoding bases agreed. 

Both users discard those rounds where Bharat chose a measurement basis different from the one Anshu used when preparing the states. This process is called **Sifting**.

#### Sifting Protocol:

- Anshu and Bharat compare the bases that they have used and keep those bits where bases matched. 
- They remove those bits from their key string where they used different bases.

<font style="color:darkorange;font-weight:bold;"> <h4>Task 1.1</h4> </font>Implement the Sifting Protocol

In [ ]:
anshu_key=[] #Anshu's register for matching rounds
bharat_key=[] #Bharat's register for matching rounds

 #your code implementing sifting

print("Anshu's key =", anshu_key)
print("Bharat's key =", bharat_key)
print(anshu_key==bharat_key) #should print 'True' if your codes are correct

#### Quantum Bit Error Correction (QBER)

At this stage, Anshu randomly chooses $\frac{1}{3}$ of the key bits to test with corresponding keys of Bharat. They exchange bits, compare and compute **Quantum Bit Error Rate (QBER)**. From the observed QBER, they can estimate the information gained by the eavesdropper during the quantum transmission stage. 

For noise-free BB84 version (noise-free implies no eaves-dropping), 

> * If **QBER Value $\neq 0$**, users abort the protocol.
> * Else, they will continue with privacy amplification step.

<h4>QBER Protocol:</h4>

- Anshu randomly chooses, say $\frac{1}{3}$ part of the final key for testing.
- Anshu and Bharat compare this part of the key bit-by-bit.
- If their bits do not match, this is considered an error.
- Error Rate is calculated as: $$QBER=\frac{\text{number of errors}}{\text{number of bits compared}}$$   
- All testing bits (one-third of the key) are discarded from their final key strings as the testing bits have been shared in the public classical channel. 

**Notice how key bits used for the protocol need to be discarded; this is because they are publicly shared and hence not secure to be used**


<font style="color:darkorange;font-weight:bold;"> <h4>Task 1.2</h4> </font>
Implement the QBER protocol and check that $QBER=0$. Also print the final secret key that can now be used in the Quantum One-Time Pad. 


In [ ]:

# your code here
       
print("QBER value =")
print("Anshu's secret key =")
print("Bharat's secret key =")

#### Privacy Amplification

What if Charlie gained *some* information about the key that Anshu and Bharat generated? So, Anshu and Bharat would not want to use the key generated earlier to encrypt their secret message. Let's suppose eavesdropper Charlie is passive i.e. he is only able to observe the communication but not alter it, then there is a direct solution to this problem based on the use of [<b>randomness extractor</b>](https://cryptography.fandom.com/wiki/Randomness_extractor)

<h4>Hashing-based extractor</h4>

From classical cryptography, **hash** is a **one-way function** $H$ that operates on an arbitrary-length message $M$ and returns a fixed-length hash value $h$. The importance of one-way hash functions is to provide a unique "fingerprint" of $M$.

$$H(M) =h$$

Some useful points:
- Given $M$, it is easy to compute $h$.
- Given $h$, it is hard to compute $M$ such that $H(M)=h$.
- Given $M$, it is hard to find another message, $M'$, such that $H(M)=H(M')$


- Given the same input, hashes always give the same output. This is an issue since if two users have the same hash, they will have the same information string which can be guessed easily.
* As a solution, random data (<b>salt</b>) is added to the input of a hash function to guarantee a unique output, even if the initial input was the same.

<font color='yellow'>**Universal hash functions are good extractors and as a consequence, confirms the security in privacy amplification.**</font>

- Randomness extractor (based on 2-universal hash functions) can be described as a system that takes a string (<b>QKD key</b>) and salt (<b>generator seed</b>), decides between $2$ hash functions and encrypts using chosen function.

**Note:** Salt length should be equal to the size of the QKD key. 

#### Privacy Amplification protocol

Our 2-universal randomness extractor will append salt and use SHA256 and SHA3 256.

- Anshu generates seed (or salt) - random bit sequence with length same as its final key and sends it to Bharat.
- If first bit of final key is $0$, Anshu will encrypt using SHA 256.
- If it is $1$, then SHA3 256 will be used.
- Anshu and Bharat generate their final key.
        
**Note:** `hashlib` functions work with strings, so you will need to convert your lists to strings.


In [ ]:
# Privacy Amplification 
# this has been implemented for you, you just have to run this cell and understand the point of this protocol

from random import randrange
import hashlib 

remain_key_anshu = anshu_key
remain_key_bharat = bharat_key

#Generating salt
salt=[]
for i in remain_key_anshu:
    a=randrange(2)
    salt.append(a)

#Adding seeds to the keys
    
remain_key_anshu.append(salt)
remain_key_bharat.append(salt)

#Converting lists to strings
str_key_anshu = ' '.join([str(elem) for elem in remain_key_anshu])
str_key_bharat = ' '.join([str(elem) for elem in remain_key_bharat])

#checking first bit to decide hash function to use

if remain_key_anshu[0]==0:
    result=hashlib.sha256(str_key_anshu.encode())
    print("Final Key Encoded using SHA256:", result.hexdigest())
else:
    result=hashlib.sha3_256(str_key_anshu.encode())
    print("Final Key Encoded using SHA3 256:", result.hexdigest())
bin(int(result.hexdigest(), 16))[2:]

## BB84 Protocol with Noise

We will modify our <b>SendState</b> function to introduce some errors. In this case implementation of the protocol above can result in $QBER \ne 0$ 

`NoisyChannel` function will help us consider more real-world QKD implementation cases.



In [1]:
#Code modified to introduce noise in communication channel

def NoisyChannel(qc1, qc2, qc1_name):
    ''' This function takes the output of a circuit qc1 (made up only of x and 
        h gates, simulate noisy quantum channel, where Pauli errors (X - bit flip; Z - phase flip
        will occur in qc2 and then initializes another circuit qc2 with introduce noise.
    ''' 
    
    # Quantum state is retrieved from qasm code of qc1
    qs = qc1.qasm().split(sep=';')[4:-1]

    # Process the code to get the instructions
    for index, instruction in enumerate(qs):
        qs[index] = instruction.lstrip()

     # Parse the instructions and apply to new circuit
    for instruction in qs:
        if instruction[0] == 'x':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.x(qc2.qregs[0][old_qr])
        elif instruction[0] == 'h':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.h(qc2.qregs[0][old_qr])
        elif instruction[0] == 'm': # exclude measuring:
            pass
        else:
            raise Exception('Unable to parse instruction')
    
    ### Introducing noise
    for instruction in qs:
        if randrange(4)<1:
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.x(qc2.qregs[0][old_qr]) #apply bit-flip error
        if randrange(4)<1:
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.z(qc2.qregs[0][old_qr]) #apply phase-flip error

<font style="color:darkorange;font-weight:bold;"> <h3>Task 2</h3> </font>

Implement the noisy version of the BB84 protocol for a $28$- bit long string upto $QBER$ calculation. Since the noise is random, run the codes repeatedly until you get $QBER \le 0.25 $. We will carryforward the protocol with this $QBER$ value.

Use the following variable names respectively for QBER value, anshu's key and bharat's key after Sifting: 
-       QBER
-       anshu_key
-       bharat_key

#### Information Reconciliation using Cascade Protocol

Before going into privacy amplification, we need to correct the erroneous bits in the generated key to generate a final error-free key for which $QBER=0$. 

Cascade Protocol uses parity check method to correct errors. Here's how Parity Check works:

> Suppose you want to send a 5 bits of information: $10110$.
> But an error occured during communication and our string became $11110$.
> How can we tell if the received information is correct or wrong?

<b>Parity bit</b> is one additional bit that we add at the end of our initial string before sending to the recepient. 

For a given set of bits, we will count how many bits we have with value '1':
- If count is even, we add parity bit $0$ at the end of our string.
- If count is odd, we add parity bit $1$.

<font style="color:darkorange;font-weight:bold;"> <h4>Task 2.1</h4> </font>

Explain with an example that parity check method cannot correct $100\%$ error in for an arbitrary message string.

            (type your answer here in maximum of 4 lines)


#### Cascade Protocol

Cascade protocol consist of several passes. 

1st PASS - Subdividing blocks:
1. Anshu and Bharat shuffle their bits by applying random permutation **that they agreed on.** (with this bits get shuffled but bit indices remain intact)
2. They subdivide their key strings to blocks of equal length. Block size is taken to be $w_1=\frac{0.73}{QBER}$ (rounded to an integer). Last block can consist of remainder bits.
3. They calculate parity bit for each block and compare parity bits.
* If parity bits match - they accept block as correct.
* If parity bits mismatch - they continue with next pass. 

2nd PASS - Bisective search: (only on blocks where parity bits didn't match)
1. Anshu and Bharat shuffle their bits based on an agreed random permutation. 
2. They divide the bad block in **half** and calculate parity bit for each half.
3. They compare **parity bits**.
* If parity bits match - they accept block as correct.
* If parity bits mismatch - they repeat the 2nd pass until they correct all errors.
##### Note:

- **Threshold:**
 
If $QBER > 0.25$, then Cascade protocol discloses lot of bits and potentially, the complete raw key. So if $QBER > 0.25$ we abort the BB84 protocol altogether because the security compromise is too much and hence we won't move to the Cascade protocol stage. 
If $QBER =0 $, then Cascade protocol can be skipped. 
If $QBER \le 0.25$, then Anshu and Bharat performs the cascade protocol


In [ ]:
#Functions necessary for the cascade protocol. 
#Check if the cascade_pass() function implements the cascade protocol as described above
import random
from random import randrange

def split(list1, n): 
    out = []
    last = 0.0
    while last < len(list1):
        out.append(list1[int(last):int(last + n)])
        last += n
    return out

def cascade_pass(A, B, n): #input key lists of Anshu (A), Bharat (B) and target block size (n)
    
    #Shuffle keys bits
    permutation = list(zip(A, B)) #this maps the index of multiple lists
    random.shuffle(permutation) #performing random permutation (note:indices still match)
    shuffledA, shuffledB = zip(*permutation) #unpacking shuffled keys of Anshu and Bharat
    
    #Split
    splitA=split(shuffledA, n) #Anshu spliting her key into blocks of length n
    splitB=split(shuffledB, n) #Bharat doing the same

    #Calculate parity
    #"correctA/B" lists will include blocks of Anshu and Bharat that has no errors
    #"errorA/B" lists includes blocks where parities mismatched (errors present)
    correctA, correctB, errorA, errorB= [], [], [], []
    sumBlocksA = [sum(block) for block in splitA]
    sumBlocksB = [sum(block) for block in splitB]
    parityA = [i %2 for i in sumBlocksA] #gives parity of Anshu's blocks
    parityB = [i %2 for i in sumBlocksB] 
    
    #Compare Parity
    for i,value in enumerate(range(len(parityA))): #comparing parity bits of Anshu and Bharat's blocks
        if parityA[i]==parityB[i]: #if parity bit matched, corresponding block added to list 'correct'
            correctA.append(splitA[i])
            correctB.append(splitB[i])
        else:
            errorA.append(splitA[i]) #if parity bits mismatched corresponding block added to list 'error'
            errorB.append(splitB[i])
        
    keyA = [item for i in correctA for item in i] #Converting correct blocks into a list
    keyB= [item for i in correctB for item in i]
    return keyA, keyB, errorA, errorB 

<font style="color:darkorange;font-weight:bold;"> <h4>Task 2.2</h4> </font>

Taking account of the threshold conditions defined above, write a program that performs the **first pass** of cascade protocol for $QBER \le 0.25$.

In [ ]:

# your code here for conditions that don't use cascade protocol, print out the keys for each case



#if 0<QBER<=0.25: #cascade protocol used if this condition holds, uncomment this condition once your QBER has been calculated
    
    # code for first pass of cascade protocol 



    # print("Anshu's key after first pass,")  # error free part of Anshu and Bharat's keys after first pass, uncomment after codes above completed
    # print("Bharat's key after second pass,")


    #Once cascade_pass function is used, we now aproximately know how many errors we have in initial key string
    #because after first pass each block in errorA and errorB lists contain an odd number of errors
    #We now can determine the final (corrected) key list length before we correct those errors (when 1 bit is left in each block)

    # The below code performing the second pass and finally correcting the error has been implemented for you, you just have to uncomment and run this after completing the code above
    # make sure to define the variables as required by the codes below
    
    # penultimatePassLength=len(anshu_key)-len(errA)  #errA and errB are the third and fourth return values of cascade_pass() function
    # print(penultimatePassLength)

    # # Bisective search at each block until corrected key length is not equal length of initial key minus error blocks number after first pass
    
    # while len(kFinalA)!=penultimatePassLength: # kfinalA and kfinalB are anshu's and bharat's error free keys after first pass. 
    #     for i, (blockA, blockB) in enumerate(zip(errA, errB)):
    #         if len(blockA)>1:
    #             secondPassA=list(blockA)# we convert block into a lists
    #             secondPassB=list(blockB)
    #             blockSize2=len(blockA)//2 #we change block size, now we will divide each block that contains an error in half
    #             corrBlockA2, corrBlockB2,  errBlockA2, errBlockB2=cascade_pass(secondPassA, secondPassB, blockSize2) #applying cascade
    #             kFinalA.extend(corrBlockA2) # adding correct bits to the key string
    #             kFinalB.extend(corrBlockB2)
    #             errA[i]=errBlockA2[0] #updating error block values
    #             errB[i]=errBlockB2[0]
    #         if len(blockA)==1: #  a side case to deal with
    #             for bit in blockA:
    #                 if bit==1:
    #                     bitA=errA[0][0]
    #                     kFinalA.append(bitA)#Anshu adds corresponding bit to her key string without change
    #                     bitB=errB[0][0]+1 # but Bharat will first correct the error by flipping the bit value 
    #                     kFinalB.append(bitB)
    #                 if bit==0:
    #                     bitA=errA[0][0]
    #                     kFinalA.append(bitA) #Anshu adds corresponding bit to her key string without change
    #                     bitB=errB[0][0]-1 # but Bharat will first correct the error by flipping the bit value 
    #                     kFinalB.append(bitB)
                        

        
    # #After previous passes we have a nested lists, to convert them:  
  
    # errorA=[item for elem in errA for item in elem]
    # errorB=[item for elem in errB for item in elem]
    
    # #Error correction step, when our error blocks contains just 1 bit (error)

    # for i, error in enumerate(zip(errorA, errorB)):
    #     bitA=int(errorA[i])
    #     bitB=int(errorB[i])
    #     if bitA==1:
    #         kFinalA.append(bitA)
    #         correctedBitB=bitB+1
    #         kFinalB.append(correctedBitB)
    #     if bitA==0:
    #         kFinalA.append(bitA)
    #         correctedBitB=bitB-1
    #         kFinalB.append(correctedBitB)
            
    # print("Final Key Anshu", kFinalA)
    # print("Final Key Bharat", kFinalB)

#### BICONF Strategy

Recall the parity bit problem. (Task $2.1$). 

Even if we see QBER $=0$ after the Cascade protocol, it doesn't mean that we can be 100% sure our key strings are identical.

#### Strategy: 
- Anshu and Bharat choose a random subset of corresponding bits from their strings.
- They compare parity bits.
- If they find a parity mismatch, they apply bisective search until they correct that error. 
- If parity bits still match after several rounds, they can conclude that their keys are identical. 

**Block Size**
- If QBER $=0$, we will use fixed block size value of $8$ bits per block. We execute $8$ rounds before we conclude that our keys are correct.
- If QBER $\ne 0$, we take the block size to be: $$b_1=\frac{4ln(2)}{3QBER}$$

**BIOCONF strategy has been implemented for you, please go through the codes and check that the final keys are error-free**

In [ ]:
# #Uncomment and run after you have implemented all the codes above, take care of the varibale names

# from numpy import log as ln

# if QBER!=0: #defining size of blocks
#     biconfBlockSize=(4*ln(2))//(3*QBER)
# if QBER==0:
#     biconfBlockSize=8
#     kFinalA=anshu_key
#     kFinalB=bharat_key

# rounds = 0 #counting rounds
# biconfError=[] #creating register for rounds with an error
# error=0 #register for found and corrected error

# while rounds!=8: #we will go through rounds and monitor if blocks with errors will be found 
#     rounds=rounds+1
#     #Creating random subsets
#     kFinalZipped=list(zip(kFinalA, kFinalB)) #maping indexes of our two lists
#     randomBlock=random.sample(list(enumerate(kFinalZipped)), int(biconfBlockSize))
    
#     #We will now need to calculate and compare parity bits for both users bits
#     sumBlockA=0
#     sumBlockB=0
#     for i in range(0,int(biconfBlockSize)):
#         sumBlockA=sumBlockA+randomBlock[i][1][0]
#         sumBlockB=sumBlockB+randomBlock[i][1][1]
#     parityA = sumBlockA%2 #then aplying mod(2) operator to our calculated sums and saving results
#     parityB = sumBlockB%2
    
#     if parityA!=parityB: #if parities of block dismatch - we bisective search to correct error before continue with next round
#         print("Error found in round:", rounds)
#         print("Applying bisective search and error correction")
#         #Applying bisective search to find and correct an error
#         while len(randomBlock)>1: #We will take our block with error and run besective search till we find bit with error
#             #Split the block
#             if len(randomBlock)%2==1: #If block size is odd
#                 half=len(randomBlock)//2+1 #Length of our first block should be half+1
#             else:
#                 half=len(randomBlock)//2
#             splitBlock=split(randomBlock, half) #spliting our block in two parts
#             for i, block in enumerate(splitBlock): #For each part
#                 sumA=0
#                 sumB=0
#                 for j in range(0,len(block)): #calculating sums 
#                     sumA=sumA+splitBlock[i][j][1][0]
#                     sumB=sumB+splitBlock[i][j][1][1]
#                 parA=sumA%2 #then calculate parities
#                 parB=sumB%2
#                 if parA==parB:
#                     pass
#                 if parA!=parB: #if parities dismatch- we update our block and run while loop again
#                     randomBlock=splitBlock[i]
#         if len(randomBlock)==1: #once we isolate the error to 1 bit
#             error=error+1
#             print("Error found in bit:", randomBlock[0][0]) #we retrieving the index of bit pair
#             errorIndex=int(randomBlock[0][0])
#             #Apply error correction at Bharats' initial key string
#         if kFinalB[errorIndex]==0:
#             kFinalB[errorIndex]=1
#         else:
#             kFinalB[errorIndex]=0
#         print("Error corrected!\n")
#     else: #If parities matched
#         pass

# print("BICONF strategy completed!\n", error, "errors found!")
# print("Final key Anshu", kFinalA)
# print("Final key Bharat", kFinalB)

With this the keys are error-free and we can proceed to the privacy amplification stage

<font style="color:darkorange;font-weight:bold;"> <h4>Task 2.3</h4> </font>

Implement privacy amplification on the keys generated to obtain the hashed version of keys that can be used in Quantum One Time pad for encryption /decryption

In [7]:
#your code here (print out the final key that can be used in One Time Pad)

### References:

- QWorld resources on [Quantum Key Distribution](https://gitlab.com/qworld/qeducation/educational-materials/self-study-modules/qkd)
- A major part of the code for `SendState()` function has been taken from [here](https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/awards/teach_me_qiskit_2018/cryptography/Cryptography.ipynb)
- Quantum Key Distribution: An Introduction with Exercises, Ramona Wolf (2021). Can be accessed [here](https://link.springer.com/book/10.1007/978-3-030-73991-1)